In [1]:
import requests
import pandas as pd
import cv2

/home/yanyu/anaconda2/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/yanyu/anaconda2/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
colors = ['red','green','blue','yellow']
DIR = "HPAv18/"
v18_url = 'http://v18.proteinatlas.org/images/'

In [3]:
imgList = pd.read_csv("notE.csv")
imgList

,Id
0,27128_1721_H8_10_cr57eab608d8d64
1,17268_1283_D1_2
2,38965_445_D6_2
3,2060_668_H10_2
4,7989_45_G11_2
5,16610_si32_F10_1
6,57202_995_F10_4
7,29937_1004_H9_2
8,29937_930_G7_2
9,23094_215_B1_1


In [4]:
len(imgList)

5888

In [ ]:
def crawl(i, num):
    img = i.split('_')
    for color in colors:
        img_path = img[0] + '/' + "_".join(img[1:]) + "_" + color + ".jpg"
        img_name = i + "_" + color + ".jpg"
        img_url = v18_url + img_path
        r = requests.get(img_url, allow_redirects=True, timeout=30)
        open(DIR + img_name, 'wb').write(r.content)
        print(num, i, 'Done!')

In [6]:
k = 0
for i in imgList['Id']: # [:5] means downloard only first 5 samples, if it works, please remove it\n",
    print(i, k)
    k += 1
    img = i.split('_')
    for color in colors:
        img_path = img[0] + '/' + "_".join(img[1:]) + "_" + color + ".jpg"
        img_name = i + "_" + color + ".jpg"
        img_url = v18_url + img_path
        r = requests.get(img_url, allow_redirects=True)
        open(DIR + img_name, 'wb').write(r.content)

27128_1721_H8_10_cr57eab608d8d64 0


ConnectionError: HTTPConnectionPool(host='v18.proteinatlas.org', port=80): Max retries exceeded with url: /images/27128/1721_H8_10_cr57eab608d8d64_red.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f317dddf5f8>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [ ]:
from multiprocessing import Pool
pool = Pool(processes=32)
for i, k in enumerate(imgList['Id']): # [:5] means downloard only first 5 samples, if it works, please remove it
    print(k)
    pool.apply_async(crawl, args=(k,i))
print('Start!')
    #     p.start()
pool.close()
pool.join()
print('DONE!!!!!!!!!!!!')

In [ ]:
!du -sh {DIR}

In [ ]:
!du -sh /HPAv18/jpg

In [ ]:
import os
import cv2
img_leak_list = []
for k, img_id in enumerate(imgList['Id']):
    try:
        for color in ['red','green','blue','yellow']:
            img = cv2.imread(os.path.join('HPAv18',img_id + "_" + color + ".jpg"))
            img = cv2.resize(img, (512,512))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(os.path.join('HPAv18_512',img_id + "_" + color + ".png" ),img)
        print('%s %s' %(k, img_id))
    except:
        img_leak_list.append(img_id)
print(len(img_leak_list))